# Scrapping Portal Inmobiliario

#### 1. Importafción de librerías

In [1]:
#importar librerías a utilizar
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from time import sleep
import random

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

from webdriver_manager.chrome import ChromeDriverManager

import warnings
warnings.filterwarnings('ignore')

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


#### 2. Scrapping de datos

A continuación se realiza un scraping de los proyectos listados en la página de Portal inmobiliario para todo Chile y se extraen todas las características relevantes publicadas

In [6]:
opts = Options()
opts.add_argument("user-agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.5938.62 Safari/537.36")
#opts.add_argument("--headless")

driver = webdriver.Chrome(
    service=Service(ChromeDriverManager().install()),
    options=opts
)

driver.get('https://www.portalinmobiliario.com/venta/departamento/proyectos')

#espera de carga de página por xx seg
wait = WebDriverWait(driver, 60)
wait.until(EC.presence_of_element_located((By.TAG_NAME, 'body')))
sleep(random.uniform(10.0, 14.0))

#Cantidad de páginas por búsqueda
#pagination = driver.find_element(By.XPATH, '//li[@class="andes-pagination__page-count"]').text
#pagination = int(pagination.split()[1])

#lista para guardar df de iteraciones
records = []

#scrapping
print("Iniciando extracción de datos...")

for page in range(1, 23):

    #nombre del proyecto
    nombre_proyectos = driver.find_elements(By.XPATH, '//div[@class="ui-search-item__title-label-grid"]')
    nombre_proyectos = [nombre_proyecto.text for nombre_proyecto in nombre_proyectos]

    #busca la moneda en la cual está publicado el anuncio
    monedas = driver.find_elements(By.XPATH, '//span[@class="andes-money-amount__currency-symbol"]')
    monedas = [ moneda.text for moneda in monedas ]
    monedas = monedas[5:]

    #busca el precio de cada publicación
    precios = driver.find_elements(By.XPATH,'//span[@class="andes-money-amount__fraction"]')
    precios = [int(precio.text.replace(".","")) if precio.text != '' else 0 for precio in precios]
    precios = precios[5:]

    #Extrae m2 del depto
    m2s = driver.find_elements(By.XPATH, '//ul[@class="ui-search-card-attributes ui-search-item__group__element ui-search-item__attributes-grid"]//li[3]')
    m2s = [m2.text for m2 in m2s]

    #extrae nro de piezas
    dormitorios = driver.find_elements(By.XPATH, '//ul[@class="ui-search-card-attributes ui-search-item__group__element ui-search-item__attributes-grid"]//li[1]')
    dormitorios = [dormitorio.text for dormitorio in dormitorios]
    
    #extrae nr de baños
    baños = driver.find_elements(By.XPATH, '//ul[@class="ui-search-card-attributes ui-search-item__group__element ui-search-item__attributes-grid"]//li[2]')
    baños = [baño.text for baño in baños]

    #unidades diponibles
    unidades_disponibles = driver.find_elements(By.XPATH, '//span[@class="ui-search-item__group__element ui-search-item__available-units-label"]')
    unidades_disponibles = [unidades.text for unidades in unidades_disponibles]

    #extrae links de proyectos
    links = driver.find_elements(By.XPATH, '//div[@class="andes-carousel-snapped__slide andes-carousel-snapped__slide--active"]//a')
    links = [link.get_attribute('href') for link in links]

    #creación de diccionario para guardar los datos extraídos
    datos = {
        "nombre_proyecto" : nombre_proyectos,
        "moneda" : monedas,
        "precio" : precios,
        "tamaño" : m2s,
        "dormitorios" : dormitorios,
        "baños" : baños,
        "unidades_disp" : unidades_disponibles,
        "link" : links
    }

    #creación de un Df a partir del diccionario
    df = pd.DataFrame(datos)

    #agregar a la lista con df el nuevo dataframe de la hoja escrapeada
    records.append(df)

    print(f"Hoja {page}, scrap completo..")


    #botón página siguiente
    try:
        next_page_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, '//a[@title="Siguiente"]'))
        )   
        
        driver.execute_script("arguments[0].click()", next_page_button)
        
        wait = WebDriverWait(driver, 20)
        wait.until(EC.presence_of_element_located((By.TAG_NAME, 'body')))
        sleep(random.uniform(10.0, 14.0))
    except:
        print("Fin scrapping.")
      

driver.close()

#concatena en un solo df los resultados de records
df = pd.concat(records)

cantidad_resultados = df.shape[0]
print(f"Scraping completo, {cantidad_resultados} resultados obtenidos.")

df.to_csv("proyectos.csv")

Iniciando extracción de datos...
Hoja 1, scrap completo..
Hoja 2, scrap completo..
Hoja 3, scrap completo..
Hoja 4, scrap completo..
Hoja 5, scrap completo..
Hoja 6, scrap completo..
Hoja 7, scrap completo..
Hoja 8, scrap completo..
Hoja 9, scrap completo..
Hoja 10, scrap completo..
Hoja 11, scrap completo..
Hoja 12, scrap completo..
Hoja 13, scrap completo..
Hoja 14, scrap completo..
Hoja 15, scrap completo..
Hoja 16, scrap completo..
Hoja 17, scrap completo..
Hoja 18, scrap completo..
Hoja 19, scrap completo..
Hoja 20, scrap completo..
Hoja 21, scrap completo..
Hoja 22, scrap completo..
Fin scrapping.
Scraping completo, 1016 resultados obtenidos.


In [7]:
df = pd.read_csv('proyectos_prueba.csv')

In [23]:
#web scraper de cada pagina de proyecto
opts = Options()
opts.add_argument("user-agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.5938.62 Safari/537.36")
opts.add_argument("--headless")

driver = webdriver.Chrome(
    service=Service(ChromeDriverManager().install()),
    options=opts
)

new_records = []

for link in df['link']:
    #abre la página dada por el link
    driver.get(link)

    #espera que cargue la pagina
    wait = WebDriverWait(driver, 1)
    wait.until(EC.presence_of_element_located((By.TAG_NAME, 'body')))
    sleep(random.uniform(0.0, 2.0))

    #extrae direccion de la pagina
    direccion = driver.find_element(By.XPATH, '//div[@class="ui-pdp-container__row ui-pdp-container__row--location"]//div//div//div//p')
    direccion = direccion.text

    #extrae estado proyecto
    estado = driver.find_element(By.XPATH, '//span[@class="ui-pdp-subtitle"]')
    estado = estado.text

    #inmobiliaria
    inmobiliaria = driver.find_element(By.XPATH, '//h3[@class="ui-pdp-color--BLACK ui-pdp-size--XSMALL ui-pdp-family--REGULAR"]')
    inmobiliaria = inmobiliaria.text

    #constructora
    try:
        constructora = driver.find_element(By.XPATH, '//div[@class="ui-vip-seller-profile ui-box-component"]//ul[2]//div[2]//div//p')
        constructora = constructora.text
    except :
        constructora = ""

    datos = {
        "direccion" : direccion, 
        "estado" : estado,
        "inmobiliaria" : inmobiliaria,
        "constructora" : constructora
    }

    #transforma el diccionario a un df
    datos_nuevos = pd.DataFrame([datos])

    #agrega el dataframe a una lista con los registros
    new_records.append(datos_nuevos)

#ciera la página
driver.close()

df = pd.concat(new_records)

#juntar el df original con los nuevos datos extraídos
#df = df.join(datos_nuevos)

#guardado de nuevos datos en un archivo csv
df.to_csv("proyectos_plus.csv")

In [8]:
df.head()

,Unnamed: 0,nombre_proyecto,moneda,precio,tamaño,dormitorios,baños,unidades_disp,link
0,0,Edificio Step,UF,2550,21 a 53 m² útiles,1 a 2 dormitorios,1 baño,129 unidades disponibles,https://www.portalinmobiliario.com/MLC-1680912...
1,1,Meneses,UF,2590,35 a 68 m² útiles,1 a 3 dormitorios,1 a 2 baños,21 unidades disponibles,https://www.portalinmobiliario.com/MLC-9830008...
2,2,Eco Irarrázaval,UF,2965,26 a 62 m² útiles,1 a 2 dormitorios,1 a 2 baños,28 unidades disponibles,https://www.portalinmobiliario.com/MLC-1600031...
3,3,Manuel Montt 2630,UF,2621,22 a 86 m² útiles,1 a 3 dormitorios,1 a 2 baños,24 unidades disponibles,https://www.portalinmobiliario.com/MLC-2063374...
4,4,Mirador 950,UF,1809,22 a 45 m² útiles,1 a 2 dormitorios,1 a 2 baños,299 unidades disponibles,https://www.portalinmobiliario.com/MLC-1390475...


#### 3. Limpieza de datos

In [10]:
df['tamaño_min'] = [ int(i.split()[0]) for i in df['tamaño'] ]
df['tamaño_max'] = [ i.split()[2] for  i in df['tamaño']]

df['dorm_min'] = [ i.split()[0] for i in df['dormitorios']]
#df['dorm_max'] = [ i.split()[2] for i in df['dormitorios']]

df['unidades_disp'] = [ i.split()[0] for i in df['unidades_disp']]


In [11]:
df['UF/m2'] = df['precio'] /df['tamaño_min']

In [20]:
df1 = df[df['precio'] > 2000]

In [21]:
df1.sort_values(by='UF/m2', ascending=True).head()

,Unnamed: 0,nombre_proyecto,moneda,precio,tamaño,dormitorios,baños,unidades_disp,link,tamaño_min,tamaño_max,dorm_min,UF/m2
595,19,Cumbres de Curauma,UF,2650,69 a 79 m² útiles,3 dormitorios,2 baños,6,https://www.portalinmobiliario.com/MLC-1135729...,69,79,3,38.405797
930,18,Aires de Limarí,UF,2200,57 a 64 m² útiles,2 a 3 dormitorios,1 a 2 baños,49,https://www.portalinmobiliario.com/MLC-9314229...,57,64,2,38.596491
1015,7,Condominio Santa Ana,UF,2300,58 a 60 m² útiles,3 dormitorios,1 a 2 baños,12,https://www.portalinmobiliario.com/MLC-1379453...,58,60,3,39.655172
840,24,Condominio Costa San Francisco II,UF,2050,50 a 55 m² útiles,2 a 3 dormitorios,1 a 2 baños,3,https://www.portalinmobiliario.com/MLC-1464930...,50,55,2,41.000000
724,4,Condominio Parque El Maitén,UF,2484,60 a 72 m² útiles,2 a 3 dormitorios,1 a 2 baños,49,https://www.portalinmobiliario.com/MLC-1379487...,60,72,2,41.400000


In [22]:
df.loc[840]['link']

'https://www.portalinmobiliario.com/MLC-1464930600-condominio-costa-san-francisco-ii-_JM#position=25&search_layout=grid&type=item&tracking_id=8fdcaff0-1cf8-4746-be01-dfdfe9f4d00c'